# Market Opportunity Analysis: Rifampicin for Prosthetic Joint Infections
## German Hospital Market – GBA Quality Reports Analysis

---

**Analysis Focus:** Identify hospitals with high prosthetic joint implant volumes and potential market opportunity for Rifampicin-based antibiotic therapy targeting periprosthetic joint infections (PJI).

**Data Source:** G-BA Strukturierte Qualitätsberichte (2017-2023)

**Target Departments:** Orthopädie, Chirurgie

---
# Section 1: Executive Summary
*→ PPT Slide: Key Findings at a Glance*

This section provides a high-level overview of the market opportunity. Detailed findings are populated after running all analyses below.

In [79]:
# Technical Setup
import sqlite3
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

# Define a helper function to display Plotly figures without nbformat dependency
def show_fig(fig):
    """Display a Plotly figure using HTML to avoid nbformat issues."""
    display(HTML(fig.to_html(include_plotlyjs='cdn', full_html=False)))

print("Plotly display: Using HTML renderer (nbformat-independent)")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:,.1f}'.format)

# Database connection
DB_FILE = Path.cwd() / "all_data_2011-2023.db"

def get_db_connection() -> sqlite3.Connection:
    return sqlite3.connect(DB_FILE, check_same_thread=False)

def run_sql(stmt: str) -> pd.DataFrame:
    with get_db_connection() as con:
        return pd.read_sql_query(stmt, con=con)

print(f"Database: {DB_FILE}")
print(f"Connection: {'OK' if DB_FILE.exists() else 'FAILED'}")

Plotly display: Using HTML renderer (nbformat-independent)
Database: /Users/tobias.setzer/python_bin/promiscuous-peacock/all_data_2011-2023.db
Connection: OK


In [80]:
# ═══════════════════════════════════════════════════════════════════════════════
# CONFIGURATION PARAMETERS
# ═══════════════════════════════════════════════════════════════════════════════

# OPS Procedure Codes (Primary Analysis)
OPS_HIP_PRIMARY = ['5-820']      # Hip endoprosthesis, primary
OPS_HIP_REVISION = ['5-821']     # Hip endoprosthesis, revision
OPS_KNEE_PRIMARY = ['5-822']     # Knee endoprosthesis, primary
OPS_KNEE_REVISION = ['5-823']    # Knee endoprosthesis, revision

OPS_ALL_RELEVANT = OPS_HIP_PRIMARY + OPS_HIP_REVISION + OPS_KNEE_PRIMARY + OPS_KNEE_REVISION

# ICD-10 Surrogate Codes (Secondary Validation)
ICD_SURROGATE = ['M16.0', 'M16.1', 'M17.0', 'M17.1']  # Primary osteoarthritis hip/knee
ICD_EXCLUSIONS = ['T84', 'Z96.6', 'M00']  # Prosthetic complications, existing prosthesis, septic arthritis

# Infection Rate Assumptions (from Briefing)
INFECTION_RATE_LOW = 0.01   # 1% - Conservative
INFECTION_RATE_MID = 0.015  # 1.5% - Mid-range
INFECTION_RATE_HIGH = 0.02  # 2% - Upper bound

# OAU Proxy Keywords
OAU_KEYWORDS = ['rifamp', 'biofilm', 'periprothetisch', 'antibiot']

# Target Department Codes
DEPT_ORTHOPEDICS = ['2300', '2309', '2315', '2316']  # Orthopädie codes
DEPT_SURGERY = ['1500', '1513', '1516', '1518', '1519', '1520', '1523']  # Chirurgie codes
DEPT_RELEVANT = DEPT_ORTHOPEDICS + DEPT_SURGERY
## Explanation on Fachabteilungscodes: https://www.vdek.com/vertragspartner/Krankenhaeuser/Datenaustausch/technische_anlagen_2019/_jcr_content/par/download_1899237012/file.res/anl2-97.pdf

# Hospital Type Classification (IK prefix for Vollversorger approximation)
# IK starting with 26 = Universitätskliniken, higher numbers often indicate larger hospitals

# Opportunity Scoring Weights
WEIGHT_VOLUME = 0.40
WEIGHT_GAP = 0.30
WEIGHT_DEPT = 0.15
WEIGHT_TYPE = 0.15

print("Configuration loaded successfully.")
print(f"\nRelevant OPS codes: {OPS_ALL_RELEVANT}")
print(f"ICD surrogate codes: {ICD_SURROGATE}")
print(f"Infection rate range: {INFECTION_RATE_LOW*100:.0f}% - {INFECTION_RATE_HIGH*100:.0f}%")

Configuration loaded successfully.

Relevant OPS codes: ['5-820', '5-821', '5-822', '5-823']
ICD surrogate codes: ['M16.0', 'M16.1', 'M17.0', 'M17.1']
Infection rate range: 1% - 2%


In [81]:
# Executive Summary - will be populated after analysis
# This cell is placed here but references data computed below

def display_executive_summary(ops_2023, trend_df, top_hospitals):
    """Display executive summary after all analyses are complete."""
    total_primary = ops_2023['hip_primary'].sum() + ops_2023['knee_primary'].sum()
    total_revision = ops_2023['hip_revision'].sum() + ops_2023['knee_revision'].sum()
    total_all = total_primary + total_revision
    
    infections_low = int(total_primary * INFECTION_RATE_LOW)
    infections_mid = int(total_primary * INFECTION_RATE_MID)
    infections_high = int(total_primary * INFECTION_RATE_HIGH)
    
    # Calculate trend
    if len(trend_df) >= 2:
        first_year = trend_df[trend_df['Berichtsjahr'] == trend_df['Berichtsjahr'].min()]['total'].values[0]
        last_year = trend_df[trend_df['Berichtsjahr'] == trend_df['Berichtsjahr'].max()]['total'].values[0]
        trend_pct = ((last_year - first_year) / first_year) * 100
    else:
        trend_pct = 0
    
    summary = f"""
╔══════════════════════════════════════════════════════════════════════════════╗
║                         EXECUTIVE SUMMARY (2023)                             ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  MARKET SIZE                                                                 ║
║  ├─ Primary Implants (Hip + Knee):     {total_primary:>8,} procedures             ║
║  ├─ Revision Procedures:               {total_revision:>8,} procedures             ║
║  └─ Total Relevant Procedures:         {total_all:>8,} procedures             ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  EXPECTED INFECTION INDEX (EII) - Primary Implants Only                      ║
║  ├─ Conservative (1.0%):               {infections_low:>8,} infections             ║
║  ├─ Mid-range (1.5%):                  {infections_mid:>8,} infections             ║
║  └─ Upper bound (2.0%):                {infections_high:>8,} infections             ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  MARKET TREND (2018-2023):             {trend_pct:>+7.1f}%                           ║
║  Hospital Count (2023):                {ops_2023['hospital_count'].sum():>8,}                           ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""
    print(summary)
    
    if top_hospitals is not None and len(top_hospitals) > 0:
        print("\n📍 TOP 5 TARGET HOSPITALS (by composite opportunity score):")
        print("─" * 70)
        for i, row in top_hospitals.head(5).iterrows():
            print(f"  {i+1}. {row['Name'][:50]:<50}")
            print(f"     Volume: {row['total_procedures']:,} | Bundesland: {row['Bundesland']}")
        print("─" * 70)

print("Executive summary function defined. Will display after analysis completion.")

Executive summary function defined. Will display after analysis completion.


---
# Section 2: Business Context
*→ PPT Slide: Why Rifampicin for PJI?*

## Periprosthetic Joint Infection (PJI) – The Clinical Challenge

**What is PJI?**
- Infection of the tissue surrounding a joint prosthesis (hip or knee replacement)
- Occurs in **1-2%** of primary joint replacements
- Higher rates in revision surgeries (up to 5-6%)

**Why is it difficult to treat?**
- Bacteria form **biofilms** on the prosthetic surface
- Biofilms protect bacteria from standard antibiotics and immune response
- Often requires multiple surgeries and prolonged antibiotic therapy

## The Rifampicin Opportunity

**Why Rifampicin?**
- One of few antibiotics with **proven biofilm penetration**
- Standard component of PJI treatment protocols (IDSA guidelines)
- Typically used in combination therapy to prevent resistance

**Market Drivers:**
1. Aging population → More joint replacements
2. Increasing revision rates → Higher infection risk
3. Antibiotic stewardship programs → Targeted therapy preferred
4. Growing awareness of biofilm-related infections

---
# Section 3: Methodology Overview
*→ PPT Slide: Our Approach*

## Data Source
**G-BA Strukturierte Qualitätsberichte** (German Hospital Quality Reports)
- Mandatory annual reports from all German hospitals
- Contains: Procedure counts (OPS), Diagnoses (ICD-10), Hospital characteristics
- Years available: 2018, 2020, 2021, 2022, 2023 (Note: 2019 missing in database)

## Primary Analysis: OPS Procedure Codes
| Code | Description | Type |
|------|-------------|------|
| 5-820 | Hip endoprosthesis | Primary |
| 5-821 | Hip endoprosthesis revision | Revision |
| 5-822 | Knee endoprosthesis | Primary |
| 5-823 | Knee endoprosthesis revision | Revision |

**Why OPS?** Direct measurement of surgical procedures performed.

## Secondary Validation: ICD-10 Surrogate
- **Inclusion:** M16.0, M16.1 (hip OA), M17.0, M17.1 (knee OA)
- **Exclusions:** T84.* (prosthetic complications), Z96.6 (existing prosthesis), M00.* (septic arthritis)

**Purpose:** Cross-validate OPS findings with diagnosis patterns.

## Key Limitation: DRG Data Unavailable
⚠️ The briefing references DRG codes I44A-C (knee) and I47A-C (hip) as gold standard.
**These are NOT available in GBA quality reports** – only at InEK/payer level.

OPS codes 5-820 to 5-823 serve as the best available proxy.

---
# Section 4: OPS Procedure Analysis (PRIMARY) – 2023 Baseline
*→ PPT Slides: Market Size by Procedure Type*

In [82]:
# Query 2023 OPS procedure volumes - National totals
ops_2023_national = run_sql("""
SELECT 
    Berichtsjahr,
    SUM(CASE WHEN OPS_301_Category LIKE '5-820%' THEN Anzahl ELSE 0 END) AS hip_primary,
    SUM(CASE WHEN OPS_301_Category LIKE '5-821%' THEN Anzahl ELSE 0 END) AS hip_revision,
    SUM(CASE WHEN OPS_301_Category LIKE '5-822%' THEN Anzahl ELSE 0 END) AS knee_primary,
    SUM(CASE WHEN OPS_301_Category LIKE '5-823%' THEN Anzahl ELSE 0 END) AS knee_revision,
    COUNT(DISTINCT IK) AS hospital_count
FROM VIEW_Krankenhaus_Prozedur
WHERE Berichtsjahr = 2023
  AND (OPS_301_Category LIKE '5-820%' 
       OR OPS_301_Category LIKE '5-821%'
       OR OPS_301_Category LIKE '5-822%' 
       OR OPS_301_Category LIKE '5-823%')
GROUP BY Berichtsjahr
""")

ops_2023_national['total_primary'] = ops_2023_national['hip_primary'] + ops_2023_national['knee_primary']
ops_2023_national['total_revision'] = ops_2023_national['hip_revision'] + ops_2023_national['knee_revision']
ops_2023_national['total'] = ops_2023_national['total_primary'] + ops_2023_national['total_revision']

print("═" * 70)
print("2023 BASELINE: National Procedure Volumes (OPS 5-820 to 5-823)")
print("═" * 70)
ops_2023_national

══════════════════════════════════════════════════════════════════════
2023 BASELINE: National Procedure Volumes (OPS 5-820 to 5-823)
══════════════════════════════════════════════════════════════════════


,Berichtsjahr,hip_primary,hip_revision,knee_primary,knee_revision,hospital_count,total_primary,total_revision,total
0,2023,262546,19539,222372,15370,974,484918,34909,519827


In [83]:
# Visualization: 2023 Procedure Mix
procedure_data = {
    'Procedure': ['Hip Primary\n(5-820)', 'Hip Revision\n(5-821)', 
                  'Knee Primary\n(5-822)', 'Knee Revision\n(5-823)'],
    'Count': [
        ops_2023_national['hip_primary'].values[0],
        ops_2023_national['hip_revision'].values[0],
        ops_2023_national['knee_primary'].values[0],
        ops_2023_national['knee_revision'].values[0]
    ],
    'Type': ['Primary', 'Revision', 'Primary', 'Revision']
}
procedure_df = pd.DataFrame(procedure_data)

fig = px.bar(
    procedure_df,
    x='Procedure',
    y='Count',
    color='Type',
    title='2023 Joint Replacement Procedures in Germany (OPS-based)',
    color_discrete_map={'Primary': '#2E86AB', 'Revision': '#E94F37'},
    text='Count'
)
fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
fig.update_layout(
    yaxis_title='Number of Procedures',
    xaxis_title='',
    showlegend=True,
    height=450
)
show_fig(fig)

# Summary statistics
print("\n📊 2023 Procedure Summary:")
print(f"   Total Primary Implants:  {ops_2023_national['total_primary'].values[0]:,}")
print(f"   Total Revisions:         {ops_2023_national['total_revision'].values[0]:,}")
print(f"   Revision Rate (Hip):     {ops_2023_national['hip_revision'].values[0] / ops_2023_national['hip_primary'].values[0] * 100:.1f}%")
print(f"   Revision Rate (Knee):    {ops_2023_national['knee_revision'].values[0] / ops_2023_national['knee_primary'].values[0] * 100:.1f}%")
print(f"   Hospitals Performing:    {ops_2023_national['hospital_count'].values[0]:,}")


📊 2023 Procedure Summary:
   Total Primary Implants:  484,918
   Total Revisions:         34,909
   Revision Rate (Hip):     7.4%
   Revision Rate (Knee):    6.9%
   Hospitals Performing:    974


---
# Section 5: Multi-Year Trend Analysis (Deep Dive)
*→ PPT Slide: Market Stability & Growth Opportunity*

**Purpose:** Validate if 2023 baseline is representative or if there's a trend

In [84]:
# Multi-year trend analysis (2018-2023)
trend_df = run_sql("""
SELECT 
    Berichtsjahr,
    SUM(CASE WHEN OPS_301_Category LIKE '5-820%' THEN Anzahl ELSE 0 END) AS hip_primary,
    SUM(CASE WHEN OPS_301_Category LIKE '5-821%' THEN Anzahl ELSE 0 END) AS hip_revision,
    SUM(CASE WHEN OPS_301_Category LIKE '5-822%' THEN Anzahl ELSE 0 END) AS knee_primary,
    SUM(CASE WHEN OPS_301_Category LIKE '5-823%' THEN Anzahl ELSE 0 END) AS knee_revision,
    COUNT(DISTINCT IK) AS hospital_count
FROM VIEW_Krankenhaus_Prozedur
WHERE Berichtsjahr BETWEEN 2017 AND 2023
  AND (OPS_301_Category LIKE '5-820%' 
       OR OPS_301_Category LIKE '5-821%'
       OR OPS_301_Category LIKE '5-822%' 
       OR OPS_301_Category LIKE '5-823%')
GROUP BY Berichtsjahr
ORDER BY Berichtsjahr
""")

trend_df['total_primary'] = trend_df['hip_primary'] + trend_df['knee_primary']
trend_df['total_revision'] = trend_df['hip_revision'] + trend_df['knee_revision']
trend_df['total'] = trend_df['total_primary'] + trend_df['total_revision']

print("═" * 80)
print("MULTI-YEAR TREND: Joint Replacement Procedures (2018-2023)")
print("═" * 80)
trend_df

════════════════════════════════════════════════════════════════════════════════
MULTI-YEAR TREND: Joint Replacement Procedures (2018-2023)
════════════════════════════════════════════════════════════════════════════════


,Berichtsjahr,hip_primary,hip_revision,knee_primary,knee_revision,hospital_count,total_primary,total_revision,total
0,2017,289527,25117,231992,17184,1054,521519,42301,563820
1,2018,292490,25269,233559,17710,1040,526049,42979,569028
2,2019,296409,24823,234534,17834,1025,530943,42657,573600
3,2020,219910,18178,166882,13059,1015,386792,31237,418029
4,2021,226498,18513,166780,13187,1009,393278,31700,424978
5,2022,245958,18194,192429,13468,987,438387,31662,470049
6,2023,262546,19539,222372,15370,974,484918,34909,519827


In [85]:
# Trend visualization
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Total Procedure Volume by Year', 'Primary vs Revision Procedures'),
    vertical_spacing=0.15
)

# Total volume trend
fig.add_trace(
    go.Scatter(
        x=trend_df['Berichtsjahr'],
        y=trend_df['total'],
        mode='lines+markers+text',
        name='Total',
        line=dict(color='#2E86AB', width=3),
        marker=dict(size=10),
        text=trend_df['total'].apply(lambda x: f'{x:,.0f}'),
        textposition='top center'
    ),
    row=1, col=1
)

# Primary vs Revision
fig.add_trace(
    go.Scatter(
        x=trend_df['Berichtsjahr'],
        y=trend_df['total_primary'],
        mode='lines+markers',
        name='Primary Implants',
        line=dict(color='#2E86AB', width=2)
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=trend_df['Berichtsjahr'],
        y=trend_df['total_revision'],
        mode='lines+markers',
        name='Revisions',
        line=dict(color='#E94F37', width=2)
    ),
    row=2, col=1
)

fig.update_layout(
    height=600,
    title_text='Joint Replacement Procedure Trends in Germany',
    showlegend=True
)
fig.update_xaxes(title_text='Year', row=2, col=1)
fig.update_yaxes(title_text='Procedures', row=1, col=1)
fig.update_yaxes(title_text='Procedures', row=2, col=1)
show_fig(fig)

In [46]:
# Trend Analysis
print("\n📈 TREND ANALYSIS:")
print("─" * 60)

# Year-over-year changes
trend_df['yoy_change'] = trend_df['total'].pct_change() * 100
trend_df['yoy_change_primary'] = trend_df['total_primary'].pct_change() * 100

# Identify COVID dip
pre_covid = trend_df[trend_df['Berichtsjahr'] == 2018]['total'].values[0] if 2018 in trend_df['Berichtsjahr'].values else None
covid_low = trend_df[trend_df['Berichtsjahr'].isin([2020, 2021])]['total'].min() if len(trend_df[trend_df['Berichtsjahr'].isin([2020, 2021])]) > 0 else None
post_covid = trend_df[trend_df['Berichtsjahr'] == 2023]['total'].values[0] if 2023 in trend_df['Berichtsjahr'].values else None

print("Year-over-Year Changes:")
for _, row in trend_df.iterrows():
    if pd.notna(row['yoy_change']):
        print(f"  {int(row['Berichtsjahr']-1)} → {int(row['Berichtsjahr'])}: {row['yoy_change']:+.1f}%")

if pre_covid and covid_low and post_covid:
    covid_impact = ((covid_low - pre_covid) / pre_covid) * 100
    recovery = ((post_covid - covid_low) / covid_low) * 100
    overall_change = ((post_covid - pre_covid) / pre_covid) * 100
    
    print(f"\n🦠 COVID Impact (2018 → lowest 2020/21): {covid_impact:+.1f}%")
    print(f"📈 Recovery (lowest → 2023):             {recovery:+.1f}%")
    print(f"📊 Overall Change (2018 → 2023):         {overall_change:+.1f}%")

# Market stability assessment
print("\n" + "═" * 60)
print("MARKET ASSESSMENT:")
print("═" * 60)

if abs(overall_change) < 5:
    print("✅ STABLE MARKET - 2023 baseline is representative")
    print("   Recommendation: Use single-year (2023) for analysis")
elif overall_change > 5:
    print("📈 GROWING MARKET - Positive trend observed")
    print("   Recommendation: Consider growth in forecasting")
else:
    print("📉 DECLINING MARKET - Negative trend observed")
    print("   Recommendation: Investigate structural factors")

print(f"\nNote: 2019 data not available in database.")


📈 TREND ANALYSIS:
────────────────────────────────────────────────────────────
Year-over-Year Changes:
  2019 → 2020: -26.5%
  2020 → 2021: +1.7%
  2021 → 2022: +10.6%
  2022 → 2023: +10.6%

🦠 COVID Impact (2018 → lowest 2020/21): -26.5%
📈 Recovery (lowest → 2023):             +24.4%
📊 Overall Change (2018 → 2023):         -8.6%

════════════════════════════════════════════════════════════
MARKET ASSESSMENT:
════════════════════════════════════════════════════════════
📉 DECLINING MARKET - Negative trend observed
   Recommendation: Investigate structural factors

Note: 2019 data not available in database.


---
# Section 6: ICD-10 Surrogate Analysis (SECONDARY)
*→ PPT Slide: Validation via Diagnosis Codes*

**Purpose:** Validate OPS findings using diagnosis patterns

In [75]:
# ICD-10 Surrogate Analysis with Exclusions
def sql_in_list(items):
    return ", ".join([f"'{i}'" for i in items])

icd_in = sql_in_list(ICD_SURROGATE)
excl_like = " OR ".join([f"ICD_10 LIKE '{p}%'" for p in ICD_EXCLUSIONS])

icd_2023_query = f"""
WITH surrogate AS (
    SELECT
        Berichtsjahr,
        SUM(Fallzahl) AS surrogate_cases,
        SUM(CASE WHEN ICD_10 IN ('M16.0', 'M16.1') THEN Fallzahl ELSE 0 END) AS hip_cases,
        SUM(CASE WHEN ICD_10 IN ('M17.0', 'M17.1') THEN Fallzahl ELSE 0 END) AS knee_cases,
        COUNT(DISTINCT IK) AS hospital_count
    FROM VIEW_Krankenhaus_Hauptdiagnosen
    WHERE Berichtsjahr = 2023
      AND ICD_10 IN ({icd_in})
    GROUP BY Berichtsjahr
),
exclusions AS (
    SELECT
        Berichtsjahr,
        SUM(Fallzahl) AS exclusion_cases,
        SUM(CASE WHEN ICD_10 LIKE 'T84%' THEN Fallzahl ELSE 0 END) AS t84_cases,
        SUM(CASE WHEN ICD_10 LIKE 'Z96.6%' THEN Fallzahl ELSE 0 END) AS z96_cases,
        SUM(CASE WHEN ICD_10 LIKE 'M00%' THEN Fallzahl ELSE 0 END) AS m00_cases
    FROM VIEW_Krankenhaus_Hauptdiagnosen
    WHERE Berichtsjahr = 2023
      AND ({excl_like})
    GROUP BY Berichtsjahr
)
SELECT
    s.Berichtsjahr,
    s.surrogate_cases,
    s.hip_cases,
    s.knee_cases,
    s.hospital_count,
    COALESCE(e.exclusion_cases, 0) AS exclusion_cases,
    COALESCE(e.t84_cases, 0) AS t84_cases,
    COALESCE(e.z96_cases, 0) AS z96_cases,
    COALESCE(e.m00_cases, 0) AS m00_cases,
    s.surrogate_cases - COALESCE(e.exclusion_cases, 0) AS net_exposure
FROM surrogate s
LEFT JOIN exclusions e ON s.Berichtsjahr = e.Berichtsjahr
"""

icd_2023 = run_sql(icd_2023_query)

print("═" * 70)
print("ICD-10 SURROGATE ANALYSIS (2023) - SECONDARY VALIDATION")
print("═" * 70)
print(f"\nInclusion Codes: {ICD_SURROGATE}")
print(f"Exclusion Prefixes: {ICD_EXCLUSIONS}")
print("\n")
icd_2023

══════════════════════════════════════════════════════════════════════
ICD-10 SURROGATE ANALYSIS (2023) - SECONDARY VALIDATION
══════════════════════════════════════════════════════════════════════

Inclusion Codes: ['M16.0', 'M16.1', 'M17.0', 'M17.1']
Exclusion Prefixes: ['T84', 'Z96.6', 'M00']




,Berichtsjahr,surrogate_cases,hip_cases,knee_cases,hospital_count,exclusion_cases,t84_cases,z96_cases,m00_cases,net_exposure
0,2023,341360,159045,182315,1062,68322,57509,0,10813,273038


In [48]:
# Compare ICD Surrogate with OPS counts
print("\n📊 OPS vs ICD COMPARISON (2023):")
print("─" * 60)

ops_total = ops_2023_national['total_primary'].values[0]
icd_total = icd_2023['net_exposure'].values[0] if len(icd_2023) > 0 else 0

print(f"OPS Primary Procedures:    {ops_total:>12,}")
print(f"ICD Net Exposure:          {icd_total:>12,}")
print(f"Difference:                {ops_total - icd_total:>+12,}")
print(f"Ratio (OPS/ICD):           {ops_total/icd_total:>12.2f}x" if icd_total > 0 else "N/A")

print("\n💡 INTERPRETATION:")
if icd_total > 0:
    ratio = ops_total / icd_total
    if 0.8 <= ratio <= 1.2:
        print("   Good correlation between OPS and ICD surrogate.")
        print("   Both methods capture similar patient populations.")
    elif ratio > 1.2:
        print("   OPS captures more cases than ICD surrogate.")
        print("   Possible reason: Not all joint replacements have M16/M17 as primary diagnosis.")
    else:
        print("   ICD surrogate shows more cases than OPS.")
        print("   Possible reason: Multiple admissions for same condition.")

print("\n⚠️  Note: ICD surrogate is a validation tool, not the primary metric.")


📊 OPS vs ICD COMPARISON (2023):
────────────────────────────────────────────────────────────
OPS Primary Procedures:         484,918
ICD Net Exposure:               273,038
Difference:                    +211,880
Ratio (OPS/ICD):                   1.78x

💡 INTERPRETATION:
   OPS captures more cases than ICD surrogate.
   Possible reason: Not all joint replacements have M16/M17 as primary diagnosis.

⚠️  Note: ICD surrogate is a validation tool, not the primary metric.


---
# Section 7: Total Addressable Market (TAM)
*→ PPT Slide: Market Sizing*

In [49]:
# TAM / EII Calculation
primary_implants = ops_2023_national['total_primary'].values[0]
revisions = ops_2023_national['total_revision'].values[0]

# Expected Infection Index (EII) - based on primary implants
eii_low = int(primary_implants * INFECTION_RATE_LOW)
eii_mid = int(primary_implants * INFECTION_RATE_MID)
eii_high = int(primary_implants * INFECTION_RATE_HIGH)

# Create TAM summary
tam_data = {
    'Metric': [
        'Primary Implants (2023)',
        'Revision Procedures (2023)',
        '─' * 30,
        'EII @ 1.0% (Conservative)',
        'EII @ 1.5% (Mid-range)',
        'EII @ 2.0% (Upper bound)'
    ],
    'Count': [
        f"{primary_implants:,}",
        f"{revisions:,}",
        '─' * 15,
        f"{eii_low:,}",
        f"{eii_mid:,}",
        f"{eii_high:,}"
    ],
    'Notes': [
        'Hip + Knee (5-820, 5-822)',
        'Hip + Knee (5-821, 5-823)',
        '',
        'Expected new PJI cases',
        'Expected new PJI cases',
        'Expected new PJI cases'
    ]
}

print("═" * 70)
print("TOTAL ADDRESSABLE MARKET (TAM) - Expected Infection Index")
print("═" * 70)
print("\nBasis: Primary implant procedures (where PJI risk applies)")
print("Infection rate: 1-2% per briefing document")
print("\n")

tam_df = pd.DataFrame(tam_data)
print(tam_df.to_string(index=False))

══════════════════════════════════════════════════════════════════════
TOTAL ADDRESSABLE MARKET (TAM) - Expected Infection Index
══════════════════════════════════════════════════════════════════════

Basis: Primary implant procedures (where PJI risk applies)
Infection rate: 1-2% per briefing document


                        Metric           Count                     Notes
       Primary Implants (2023)         484,918 Hip + Knee (5-820, 5-822)
    Revision Procedures (2023)          34,909 Hip + Knee (5-821, 5-823)
────────────────────────────── ───────────────                          
     EII @ 1.0% (Conservative)           4,849    Expected new PJI cases
        EII @ 1.5% (Mid-range)           7,273    Expected new PJI cases
      EII @ 2.0% (Upper bound)           9,698    Expected new PJI cases


In [86]:
# TAM Visualization
fig = go.Figure()

# Funnel visualization
fig.add_trace(go.Funnel(
    y=['Primary Implants', 'EII @ 2.0%', 'EII @ 1.5%', 'EII @ 1.0%'],
    x=[primary_implants, eii_high, eii_mid, eii_low],
    textinfo='value+percent initial',
    marker=dict(color=['#2E86AB', '#E94F37', '#F39C12', '#27AE60'])
))

fig.update_layout(
    title='Market Funnel: From Implants to Expected Infections',
    height=400
)
show_fig(fig)

print("\n📊 KEY INSIGHT:")
print(f"   Even at conservative 1% infection rate, ~{eii_low:,} patients/year")
print(f"   represent potential Rifampicin treatment opportunities.")


📊 KEY INSIGHT:
   Even at conservative 1% infection rate, ~4,849 patients/year
   represent potential Rifampicin treatment opportunities.


---
# Section 8: OAU Proxy Analysis
*→ PPT Slide: Antibiotic Awareness Indicator (with caveats)*

## ⚠️ CRITICAL CAVEAT

**The OAU (Observed Antibiotic Use) metric is a TEXT-BASED PROXY only.**

- NO actual Rifampicin usage data exists in GBA quality reports
- NO DDD (Defined Daily Doses) or therapy days available
- This proxy searches hospital service offering texts for keywords
- **Can only indicate awareness/capability, NOT actual consumption**

The derived AAI (Antibiotic Adequacy Index) is **INDICATIVE ONLY**.

In [51]:
# OAU Proxy Analysis
def get_oau_proxy_df(year: int, keywords: list[str]) -> pd.DataFrame:
    keyword_clause = " OR ".join([
        f"LOWER(COALESCE(mla.Bezeichnung, '')) LIKE '%{kw}%'"
        f" OR LOWER(COALESCE(mla.Erlaeuterungen, '')) LIKE '%{kw}%'"
        for kw in keywords
    ])
    stmt = f"""
SELECT
    v.Berichtsjahr,
    v.IK,
    v.Name,
    MIN(v.Ort) AS Ort,
    MIN(v.Postleitzahl) AS Postleitzahl,
    MIN(v.geo_Bundesland) AS Bundesland,
    AVG(v.geo_Lat) AS Latitude,
    AVG(v.geo_Lon) AS Longitude,
    COUNT(DISTINCT mla.ID) AS antibiotic_mention_count
FROM VIEW_Krankenhaus_GEO v
JOIN REL_Qualitaetsbericht_Organisationseinheit_Fachabteilung rqo
  ON rqo.Qualitaetsbericht_ID = v.Qualitaetsbericht_ID
JOIN REL_Organisationseinheit_Fachabteilung_Medizinisches_Leistungsangebot rom
  ON rom.Organisationseinheit_Fachabteilung_ID = rqo.Organisationseinheit_Fachabteilung_ID
JOIN Medizinisches_Leistungsangebot mla
  ON mla.ID = rom.Medizinisches_Leistungsangebot_ID
WHERE v.Berichtsjahr = {year}
  AND ({keyword_clause})
GROUP BY v.Berichtsjahr, v.IK, v.Name;
"""
    return run_sql(stmt)

oau_df = get_oau_proxy_df(2023, OAU_KEYWORDS)

print("═" * 70)
print("OAU PROXY ANALYSIS (2023) - Text-Based Antibiotic Awareness")
print("═" * 70)
print(f"\nKeywords searched: {OAU_KEYWORDS}")
print(f"Hospitals with matches: {len(oau_df)}")
print(f"\n⚠️  REMINDER: This is a PROXY metric, not actual usage data!")
print("\n")
oau_df.sort_values('antibiotic_mention_count', ascending=False).head(10)

══════════════════════════════════════════════════════════════════════
OAU PROXY ANALYSIS (2023) - Text-Based Antibiotic Awareness
══════════════════════════════════════════════════════════════════════

Keywords searched: ['rifamp', 'biofilm', 'periprothetisch', 'antibiot']
Hospitals with matches: 113

⚠️  REMINDER: This is a PROXY metric, not actual usage data!




,Berichtsjahr,IK,Name,Ort,Postleitzahl,Bundesland,Latitude,Longitude,antibiotic_mention_count
101,2023,261401030,Universitätsklinikum Carl Gustav Carus Dresden...,Dresden,01307,Sachsen,51.1,13.8,12
1,2023,260102343,UNIVERSITÄTSKLINIKUM Schleswig-Holstein,Lübeck,23538,Schleswig-Holstein,53.8,10.7,6
87,2023,261101015,Charité - Universitätsmedizin Berlin,Berlin,10117,Deutschland,52.5,13.4,6
67,2023,260820569,Universitätsklinikum Mannheim GmbH,Mannheim,68167,Baden-Württemberg,49.5,8.5,5
78,2023,260920069,Klinikum Passau,Passau,94032,Bayern,48.6,13.4,5
74,2023,260913195,Klinikum rechts der Isar der Technischen Unive...,München,81675,Bayern,48.1,11.6,5
59,2023,260730161,Universitätsmedizin der Johannes Gutenberg-Uni...,Mainz,55131,Rheinland-Pfalz,50.0,8.3,4
55,2023,260700109,Gemeinschaftsklinikum Mittelrhein gGmbH,Koblenz,56073,Rheinland-Pfalz,50.4,7.6,4
79,2023,260930608,Universitätsklinikum Regensburg,Regensburg,93053,Bayern,49.0,12.1,4
82,2023,260950011,ANregiomed,Ansbach,91522,Bayern,49.3,10.6,4


In [52]:
# OAU Distribution
fig = px.histogram(
    oau_df,
    x='antibiotic_mention_count',
    nbins=30,
    title='Distribution of Antibiotic Keyword Mentions (OAU Proxy)',
    labels={'antibiotic_mention_count': 'Number of Keyword Mentions'},
    color_discrete_sequence=['#2E86AB']
)
fig.update_layout(
    yaxis_title='Number of Hospitals',
    height=400,
    annotations=[dict(
        x=0.95, y=0.95,
        xref='paper', yref='paper',
        text='⚠️ Proxy metric only',
        showarrow=False,
        font=dict(size=12, color='red'),
        bgcolor='lightyellow'
    )]
)
show_fig(fig)

print(f"\n📊 OAU Proxy Summary:")
print(f"   Hospitals with ANY mention: {len(oau_df[oau_df['antibiotic_mention_count'] > 0])}")
print(f"   Mean mentions: {oau_df['antibiotic_mention_count'].mean():.1f}")
print(f"   Max mentions: {oau_df['antibiotic_mention_count'].max()}")


📊 OAU Proxy Summary:
   Hospitals with ANY mention: 113
   Mean mentions: 1.7
   Max mentions: 12


---
# Section 9: Hospital Opportunity Ranking
*→ PPT Slides: Target Hospital Lists*

**Composite Scoring:**
- Volume (40%): Procedure counts
- Gap Proxy (30%): Low OAU relative to volume
- Departments (15%): Presence of Orthopädie/Chirurgie
- Hospital Type (15%): Vollversorger approximation

In [53]:
# Hospital-level OPS data (2023)
hospital_ops_df = run_sql("""
SELECT 
    IK,
    Name,
    MIN(Ort) AS Ort,
    MIN(Postleitzahl) AS Postleitzahl,
    MIN(geo_Bundesland) AS Bundesland,
    AVG(geo_Lat) AS Latitude,
    AVG(geo_Lon) AS Longitude,
    SUM(CASE WHEN OPS_301_Category LIKE '5-820%' THEN Anzahl ELSE 0 END) AS hip_primary,
    SUM(CASE WHEN OPS_301_Category LIKE '5-821%' THEN Anzahl ELSE 0 END) AS hip_revision,
    SUM(CASE WHEN OPS_301_Category LIKE '5-822%' THEN Anzahl ELSE 0 END) AS knee_primary,
    SUM(CASE WHEN OPS_301_Category LIKE '5-823%' THEN Anzahl ELSE 0 END) AS knee_revision
FROM VIEW_Krankenhaus_Prozedur
WHERE Berichtsjahr = 2023
  AND (OPS_301_Category LIKE '5-820%' 
       OR OPS_301_Category LIKE '5-821%'
       OR OPS_301_Category LIKE '5-822%' 
       OR OPS_301_Category LIKE '5-823%')
GROUP BY IK, Name
HAVING (hip_primary + hip_revision + knee_primary + knee_revision) > 0
""")

hospital_ops_df['total_primary'] = hospital_ops_df['hip_primary'] + hospital_ops_df['knee_primary']
hospital_ops_df['total_revision'] = hospital_ops_df['hip_revision'] + hospital_ops_df['knee_revision']
hospital_ops_df['total_procedures'] = hospital_ops_df['total_primary'] + hospital_ops_df['total_revision']
hospital_ops_df['revision_rate'] = hospital_ops_df['total_revision'] / (hospital_ops_df['total_primary'] + 0.001)

print(f"Hospitals performing joint replacements (2023): {len(hospital_ops_df)}")
hospital_ops_df.sort_values('total_procedures', ascending=False).head(10)

Hospitals performing joint replacements (2023): 1005


,IK,Name,Ort,Postleitzahl,Bundesland,Latitude,Longitude,hip_primary,hip_revision,knee_primary,knee_revision,total_primary,total_revision,total_procedures,revision_rate
42,260200320,Helios ENDO-Klinik Hamburg,Hamburg,22767,Deutschland,53.6,10.0,2100,504,2070,426,4170,930,5100,0.2
649,260911353,Sana Klinik München GmbH,München,81369,Bayern,48.1,11.5,2402,105,2115,104,4517,209,4726,0.0
578,260822195,ARCUS Klinik Pforzheim,Pforzheim,75179,Baden-Württemberg,48.9,8.7,1711,38,2720,120,4431,158,4589,0.0
761,260960796,Orthopädisches Krankenhaus Schloss Werneck,Werneck,97440,Bayern,50.0,10.1,1577,63,2841,105,4418,168,4586,0.0
522,260810146,Diakonie-Klinikum Stuttgart,Stuttgart,70176,Baden-Württemberg,48.8,9.2,1770,197,2036,204,3806,401,4207,0.1
21,260101809,Schön Klinik Neustadt,Neustadt in Holstein,23730,Schleswig-Holstein,54.1,10.8,1638,113,2134,70,3772,183,3955,0.0
861,261201232,Sana Kliniken Sommerfeld,Kremmen,16766,Brandenburg,52.8,13.0,1654,143,1993,101,3647,244,3891,0.1
983,261600304,Waldkliniken Eisenberg,Eisenberg,07607,Thüringen,51.0,11.9,1612,115,1532,92,3144,207,3351,0.1
25,260102150,Klinik Manhagen,Großhansdorf,22927,Schleswig-Holstein,53.7,10.3,1266,12,1871,57,3137,69,3206,0.0
630,260910205,Barmherzige Brüder Krankenhaus München,München,80639,Bayern,48.2,11.5,1523,219,1126,112,2649,331,2980,0.1


In [54]:
# Department validation - check for Orthopädie/Chirurgie
dept_codes_str = ", ".join([f"'{c}'" for c in DEPT_RELEVANT])

hospital_depts_df = run_sql(f"""
SELECT DISTINCT
    vkf.IK,
    1 AS has_relevant_dept
FROM VIEW_Krankenhaus_Fachabteilung vkf
JOIN REL_Organisationseinheit_Fachabteilung_Fachabteilungsschluessel rof
    ON rof.Organisationseinheit_Fachabteilung_ID = vkf.ID_OE
JOIN Fachabteilungsschluessel fs
    ON fs.ID = rof.Fachabteilungsschluessel_ID
WHERE vkf.Berichtsjahr = 2023
  AND SUBSTR(fs.FA_Schluessel, 1, 4) IN ({dept_codes_str})
""")

print(f"Hospitals with Orthopädie/Chirurgie departments: {len(hospital_depts_df)}")

Hospitals with Orthopädie/Chirurgie departments: 1039


In [55]:
# Hospital Type Classification (IK prefix approximation)
# IK prefix patterns for German hospitals:
# - 26xxxxx: University hospitals (Universitätskliniken)
# - IK length and pattern can indicate hospital size/type

def classify_hospital_type(ik):
    """Approximate hospital type from IK number."""
    ik_str = str(ik)
    if ik_str.startswith('26'):
        return 'Universitätsklinik'
    # Other patterns could be added based on regional knowledge
    return 'Allgemeinkrankenhaus'

hospital_ops_df['hospital_type'] = hospital_ops_df['IK'].apply(classify_hospital_type)
hospital_ops_df['is_vollversorger'] = hospital_ops_df['total_procedures'] >= 500  # High-volume proxy

print("Hospital Type Distribution:")
print(hospital_ops_df['hospital_type'].value_counts())
print(f"\nHigh-volume hospitals (≥500 procedures): {hospital_ops_df['is_vollversorger'].sum()}")

Hospital Type Distribution:
hospital_type
Universitätsklinik    1005
Name: count, dtype: int64

High-volume hospitals (≥500 procedures): 343


In [56]:
# Merge all hospital data and create composite score
hospital_df = hospital_ops_df.merge(
    oau_df[['IK', 'antibiotic_mention_count']],
    on='IK',
    how='left'
)
hospital_df['antibiotic_mention_count'] = hospital_df['antibiotic_mention_count'].fillna(0)

hospital_df = hospital_df.merge(
    hospital_depts_df,
    on='IK',
    how='left'
)
hospital_df['has_relevant_dept'] = hospital_df['has_relevant_dept'].fillna(0).astype(int)

# Normalize scores
hospital_df['volume_score'] = hospital_df['total_procedures'] / hospital_df['total_procedures'].max()

# Gap score: inverse of OAU (low OAU = high gap = opportunity)
oau_max = hospital_df['antibiotic_mention_count'].max()
if oau_max > 0:
    hospital_df['oau_norm'] = hospital_df['antibiotic_mention_count'] / oau_max
else:
    hospital_df['oau_norm'] = 0
hospital_df['gap_score'] = 1 - hospital_df['oau_norm']  # Invert: low OAU = high opportunity

# Department score
hospital_df['dept_score'] = hospital_df['has_relevant_dept']

# Type score (Vollversorger/high-volume)
hospital_df['type_score'] = hospital_df['is_vollversorger'].astype(int)

# Composite opportunity score
hospital_df['opportunity_score'] = (
    WEIGHT_VOLUME * hospital_df['volume_score'] +
    WEIGHT_GAP * hospital_df['gap_score'] +
    WEIGHT_DEPT * hospital_df['dept_score'] +
    WEIGHT_TYPE * hospital_df['type_score']
)

# EII calculation per hospital
hospital_df['EII_low'] = hospital_df['total_primary'] * INFECTION_RATE_LOW
hospital_df['EII_mid'] = hospital_df['total_primary'] * INFECTION_RATE_MID
hospital_df['EII_high'] = hospital_df['total_primary'] * INFECTION_RATE_HIGH

# Sort by opportunity score
hospital_ranked = hospital_df.sort_values('opportunity_score', ascending=False).reset_index(drop=True)

print("═" * 80)
print("HOSPITAL OPPORTUNITY RANKING (Top 20)")
print("═" * 80)
print(f"\nScoring Weights: Volume={WEIGHT_VOLUME}, Gap={WEIGHT_GAP}, Dept={WEIGHT_DEPT}, Type={WEIGHT_TYPE}")
print("\n")

display_cols = ['IK', 'Name', 'Bundesland', 'total_procedures', 'EII_mid', 
                'antibiotic_mention_count', 'opportunity_score']
hospital_ranked[display_cols].head(20)

════════════════════════════════════════════════════════════════════════════════
HOSPITAL OPPORTUNITY RANKING (Top 20)
════════════════════════════════════════════════════════════════════════════════

Scoring Weights: Volume=0.4, Gap=0.3, Dept=0.15, Type=0.15




,IK,Name,Bundesland,total_procedures,EII_mid,antibiotic_mention_count,opportunity_score
0,260200320,Helios ENDO-Klinik Hamburg,Deutschland,5100,62.5,0.0,1.0
1,260911353,Sana Klinik München GmbH,Bayern,4726,67.8,0.0,1.0
2,260822195,ARCUS Klinik Pforzheim,Baden-Württemberg,4589,66.5,0.0,1.0
3,260960796,Orthopädisches Krankenhaus Schloss Werneck,Bayern,4586,66.3,0.0,1.0
4,260810146,Diakonie-Klinikum Stuttgart,Baden-Württemberg,4207,57.1,0.0,0.9
5,260101809,Schön Klinik Neustadt,Schleswig-Holstein,3955,56.6,0.0,0.9
6,261201232,Sana Kliniken Sommerfeld,Brandenburg,3891,54.7,0.0,0.9
7,261600304,Waldkliniken Eisenberg,Thüringen,3351,47.2,0.0,0.9
8,260102150,Klinik Manhagen,Schleswig-Holstein,3206,47.1,0.0,0.9
9,260910205,Barmherzige Brüder Krankenhaus München,Bayern,2980,39.7,0.0,0.8


In [57]:
# Create Tiered Target Lists
print("═" * 80)
print("TIERED TARGET LISTS")
print("═" * 80)

# Tier 1: High volume + Vollversorger + relevant depts + low AAI
tier1 = hospital_ranked[
    (hospital_ranked['total_procedures'] >= 500) &
    (hospital_ranked['has_relevant_dept'] == 1) &
    (hospital_ranked['oau_norm'] < 0.3)
].head(20)

# Tier 2: Medium volume + relevant depts + low AAI
tier2 = hospital_ranked[
    (hospital_ranked['total_procedures'] >= 200) &
    (hospital_ranked['total_procedures'] < 500) &
    (hospital_ranked['has_relevant_dept'] == 1) &
    (hospital_ranked['oau_norm'] < 0.5)
].head(20)

# Tier 3: Reference centers (high revision rate - complex cases)
tier3 = hospital_ranked[
    (hospital_ranked['total_revision'] >= 50) &
    (hospital_ranked['revision_rate'] > 0.1)
].sort_values('total_revision', ascending=False).head(20)

print(f"\n🎯 TIER 1 - Priority Targets (High Volume + Gap): {len(tier1)} hospitals")
print("─" * 60)
if len(tier1) > 0:
    print(tier1[['Name', 'Bundesland', 'total_procedures', 'EII_mid']].head(10).to_string(index=False))

print(f"\n🎯 TIER 2 - Secondary Targets (Medium Volume): {len(tier2)} hospitals")
print("─" * 60)
if len(tier2) > 0:
    print(tier2[['Name', 'Bundesland', 'total_procedures', 'EII_mid']].head(10).to_string(index=False))

print(f"\n🎯 TIER 3 - Reference Centers (High Revision): {len(tier3)} hospitals")
print("─" * 60)
if len(tier3) > 0:
    print(tier3[['Name', 'Bundesland', 'total_revision', 'revision_rate']].head(10).to_string(index=False))

════════════════════════════════════════════════════════════════════════════════
TIERED TARGET LISTS
════════════════════════════════════════════════════════════════════════════════

🎯 TIER 1 - Priority Targets (High Volume + Gap): 20 hospitals
────────────────────────────────────────────────────────────
                                      Name         Bundesland  total_procedures  EII_mid
                Helios ENDO-Klinik Hamburg        Deutschland              5100     62.5
                  Sana Klinik München GmbH             Bayern              4726     67.8
                    ARCUS Klinik Pforzheim  Baden-Württemberg              4589     66.5
Orthopädisches Krankenhaus Schloss Werneck             Bayern              4586     66.3
               Diakonie-Klinikum Stuttgart  Baden-Württemberg              4207     57.1
                     Schön Klinik Neustadt Schleswig-Holstein              3955     56.6
                  Sana Kliniken Sommerfeld        Brandenburg          

---
# Section 10: Geographic Analysis
*→ PPT Slides: Regional Market View*

In [58]:
# Regional/Bundesland Analysis
regional_df = hospital_df.groupby('Bundesland', as_index=False).agg({
    'total_procedures': 'sum',
    'total_primary': 'sum',
    'total_revision': 'sum',
    'EII_mid': 'sum',
    'IK': 'count',
    'opportunity_score': 'mean'
}).rename(columns={'IK': 'hospital_count'})

regional_df['procedures_per_hospital'] = regional_df['total_procedures'] / regional_df['hospital_count']
regional_df = regional_df.sort_values('total_procedures', ascending=False)

print("═" * 80)
print("REGIONAL MARKET ANALYSIS (by Bundesland)")
print("═" * 80)
regional_df

════════════════════════════════════════════════════════════════════════════════
REGIONAL MARKET ANALYSIS (by Bundesland)
════════════════════════════════════════════════════════════════════════════════


,Bundesland,total_procedures,total_primary,total_revision,EII_mid,hospital_count,opportunity_score,procedures_per_hospital
8,Nordrhein-Westfalen,103333,96281,7052,"1,444.2",212,0.5,487.4
1,Bayern,94989,88784,6205,"1,331.8",167,0.5,568.8
0,Baden-Württemberg,64256,60124,4132,901.9,100,0.6,642.6
7,Niedersachsen,47557,44299,3258,664.5,101,0.5,470.9
4,Deutschland,38854,35394,3460,530.9,69,0.5,563.1
5,Hessen,35213,32923,2290,493.8,79,0.5,445.7
11,Sachsen,28926,26724,2202,400.9,54,0.6,535.7
13,Schleswig-Holstein,22066,20810,1256,312.1,29,0.6,760.9
9,Rheinland-Pfalz,21492,20340,1152,305.1,53,0.5,405.5
14,Thüringen,16785,15762,1023,236.4,30,0.5,559.5


In [59]:
# Regional visualization
fig = px.bar(
    regional_df,
    x='Bundesland',
    y='total_procedures',
    color='EII_mid',
    title='Joint Replacement Procedures by Bundesland (2023)',
    labels={'total_procedures': 'Total Procedures', 'EII_mid': 'Expected Infections (1.5%)'},
    color_continuous_scale='Reds'
)
fig.update_layout(xaxis_tickangle=-45, height=500)
show_fig(fig)

In [60]:
# Hospital Map Visualization
map_df = hospital_ranked[hospital_ranked['Latitude'].notna()].copy()

if len(map_df) > 0:
    fig = px.scatter_mapbox(
        map_df,
        lat='Latitude',
        lon='Longitude',
        size='total_procedures',
        color='opportunity_score',
        hover_name='Name',
        hover_data=['Bundesland', 'total_procedures', 'EII_mid'],
        color_continuous_scale='RdYlGn_r',
        size_max=30,
        zoom=5,
        center={'lat': 51.1657, 'lon': 10.4515},
        title='Hospital Market Opportunity Map (Germany)'
    )
    fig.update_layout(
        mapbox_style='carto-positron',
        height=600
    )
    show_fig(fig)
else:
    print("No geographic coordinates available for map visualization.")

In [61]:
# Identify Regional Hotspots
print("\n🔥 REGIONAL OPPORTUNITY HOTSPOTS")
print("═" * 60)

# Highest volume regions
top_volume = regional_df.nlargest(3, 'total_procedures')
print("\nTop 3 by Volume:")
for _, row in top_volume.iterrows():
    print(f"  • {row['Bundesland']}: {row['total_procedures']:,} procedures, {row['hospital_count']} hospitals")

# Highest opportunity score regions
top_opportunity = regional_df.nlargest(3, 'opportunity_score')
print("\nTop 3 by Average Opportunity Score:")
for _, row in top_opportunity.iterrows():
    print(f"  • {row['Bundesland']}: Score {row['opportunity_score']:.3f}")

# Highest concentration (procedures per hospital)
top_concentration = regional_df.nlargest(3, 'procedures_per_hospital')
print("\nTop 3 by Concentration (procedures/hospital):")
for _, row in top_concentration.iterrows():
    print(f"  • {row['Bundesland']}: {row['procedures_per_hospital']:.0f} procedures/hospital")


🔥 REGIONAL OPPORTUNITY HOTSPOTS
════════════════════════════════════════════════════════════

Top 3 by Volume:
  • Nordrhein-Westfalen: 103,333 procedures, 212 hospitals
  • Bayern: 94,989 procedures, 167 hospitals
  • Baden-Württemberg: 64,256 procedures, 100 hospitals

Top 3 by Average Opportunity Score:
  • Schleswig-Holstein: Score 0.570
  • Baden-Württemberg: Score 0.559
  • Sachsen: Score 0.553

Top 3 by Concentration (procedures/hospital):
  • Schleswig-Holstein: 761 procedures/hospital
  • Baden-Württemberg: 643 procedures/hospital
  • Bayern: 569 procedures/hospital


---
# Section 11: Recommendations & Next Steps
*→ PPT Slide: Recommended Actions*

In [62]:
# Generate Recommendations
print("═" * 80)
print("RECOMMENDATIONS & NEXT STEPS")
print("═" * 80)

print("""
📋 PRIORITY ACTIONS FOR FIELD TEAM:

1. IMMEDIATE TARGETS (Tier 1)
   - Focus on high-volume hospitals with identified gap in antibiotic awareness
   - Schedule Medical Education sessions on biofilm management
   - Present Rifampicin clinical evidence and IDSA guidelines

2. REGIONAL STRATEGY
   - Prioritize Bundesländer with highest procedure volumes
   - Consider regional medical conferences for awareness campaigns
   - Identify Key Opinion Leaders in orthopedic infectious disease

3. REFERENCE CENTER ENGAGEMENT (Tier 3)
   - High-revision hospitals often handle complex PJI cases
   - These centers may influence treatment protocols regionally
   - Consider clinical collaboration or research partnerships

4. DATA VALIDATION
   - Cross-reference with IQVIA/Insight Health for actual Rx data
   - Consider hospital surveys to validate awareness proxy
   - Engage with hospital pharmacies for consumption data

⚠️  IMPORTANT CAVEATS:
   - OAU proxy indicates awareness, NOT actual usage
   - Infection rates are estimates (1-2% from literature)
   - Hospital-level data may have reporting variations
""")

════════════════════════════════════════════════════════════════════════════════
RECOMMENDATIONS & NEXT STEPS
════════════════════════════════════════════════════════════════════════════════

📋 PRIORITY ACTIONS FOR FIELD TEAM:

1. IMMEDIATE TARGETS (Tier 1)
   - Focus on high-volume hospitals with identified gap in antibiotic awareness
   - Schedule Medical Education sessions on biofilm management
   - Present Rifampicin clinical evidence and IDSA guidelines

2. REGIONAL STRATEGY
   - Prioritize Bundesländer with highest procedure volumes
   - Consider regional medical conferences for awareness campaigns
   - Identify Key Opinion Leaders in orthopedic infectious disease

3. REFERENCE CENTER ENGAGEMENT (Tier 3)
   - High-revision hospitals often handle complex PJI cases
   - These centers may influence treatment protocols regionally
   - Consider clinical collaboration or research partnerships

4. DATA VALIDATION
   - Cross-reference with IQVIA/Insight Health for actual Rx data
   - Con

---
# Section 12: Limitations & Data Gaps
*→ PPT Slide: What We Cannot Answer*

In [63]:
# Limitations and Data Gaps
limitations_data = {
    'Briefing Requirement': [
        'DRG codes I44/I47 (gold standard)',
        'Patient age filter (≥50/55)',
        'Actual antibiotic consumption (DDD)',
        'Rifampicin therapy days',
        'Outpatient follow-up treatment',
        'Case-level infection linkage',
        'Antibiotic Adequacy Index (true)',
        'Hospital case mix adjustment',
        'Supply level (Versorgungsstufe)'
    ],
    'Available?': [
        '❌ No',
        '❌ No',
        '❌ No',
        '❌ No',
        '❌ No',
        '❌ No',
        '❌ No',
        '⚠️ Partial',
        '⚠️ Partial'
    ],
    'Gap Description': [
        'Used OPS 5-820 to 5-823 as proxy',
        'No demographics in GBA data (Destatis has national aggregates)',
        'No pharmacy data in quality reports',
        'No treatment duration data',
        'Only inpatient data available',
        'Cannot link procedures to infections',
        'Only text-based proxy available',
        'Revision rate as complexity proxy only',
        'IK prefix approximation only'
    ],
    'Potential Data Source': [
        'InEK DRG database',
        'Destatis (Genesis 23141)',
        'IQVIA / Insight Health',
        'Hospital pharmacy data',
        'AOK/Barmer claims',
        'Hospital infection registries',
        'Hospital surveys',
        'Case mix index from InEK',
        'Hospital directories'
    ]
}

limitations_df = pd.DataFrame(limitations_data)

print("═" * 90)
print("DATA GAPS: Briefing Requirements vs. Available Data")
print("═" * 90)
print("\n")
print(limitations_df.to_string(index=False))

══════════════════════════════════════════════════════════════════════════════════════════
DATA GAPS: Briefing Requirements vs. Available Data
══════════════════════════════════════════════════════════════════════════════════════════


               Briefing Requirement Available?                                                Gap Description         Potential Data Source
  DRG codes I44/I47 (gold standard)       ❌ No                               Used OPS 5-820 to 5-823 as proxy             InEK DRG database
        Patient age filter (≥50/55)       ❌ No No demographics in GBA data (Destatis has national aggregates)      Destatis (Genesis 23141)
Actual antibiotic consumption (DDD)       ❌ No                            No pharmacy data in quality reports        IQVIA / Insight Health
            Rifampicin therapy days       ❌ No                                     No treatment duration data        Hospital pharmacy data
     Outpatient follow-up treatment       ❌ No                  

In [64]:
# Backlog Items for Future Enrichment
print("\n" + "═" * 80)
print("BACKLOG: Future Data Enrichment Opportunities")
print("═" * 80)

print("""
📌 DESTATIS AGE DISTRIBUTION (Backlog)
   Source: https://www-genesis.destatis.de/datenbank/online/statistic/23141
   - National age distribution for ICD-10 and OPS codes
   - Caveat: Aggregated data (not hospital-level)
   - Use case: Validate age distribution assumptions (≥50/55 filter)
   - Status: Available for future integration

📌 INEK DRG DATA (Requires Access)
   - Hospital-level DRG assignments not publicly available
   - Would provide gold-standard I44/I47 procedure counts
   - Requires institutional data access agreement

📌 CLAIMS DATA PARTNERSHIPS
   - AOK/Barmer Routinedaten
   - Would enable actual treatment pathway analysis
   - Requires payer partnership

📌 PHARMACY DATA (Commercial)
   - IQVIA / Insight Health
   - Actual Rifampicin dispensing by hospital
   - Standard commercial data product
""")


════════════════════════════════════════════════════════════════════════════════
BACKLOG: Future Data Enrichment Opportunities
════════════════════════════════════════════════════════════════════════════════

📌 DESTATIS AGE DISTRIBUTION (Backlog)
   Source: https://www-genesis.destatis.de/datenbank/online/statistic/23141
   - National age distribution for ICD-10 and OPS codes
   - Caveat: Aggregated data (not hospital-level)
   - Use case: Validate age distribution assumptions (≥50/55 filter)
   - Status: Available for future integration

📌 INEK DRG DATA (Requires Access)
   - Hospital-level DRG assignments not publicly available
   - Would provide gold-standard I44/I47 procedure counts
   - Requires institutional data access agreement

📌 CLAIMS DATA PARTNERSHIPS
   - AOK/Barmer Routinedaten
   - Would enable actual treatment pathway analysis
   - Requires payer partnership

📌 PHARMACY DATA (Commercial)
   - IQVIA / Insight Health
   - Actual Rifampicin dispensing by hospital
   - Sta

---
# Final Executive Summary

This section displays the complete executive summary after all analyses have been computed.

In [65]:
# Display Final Executive Summary
display_executive_summary(ops_2023_national, trend_df, hospital_ranked)


╔══════════════════════════════════════════════════════════════════════════════╗
║                         EXECUTIVE SUMMARY (2023)                             ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  MARKET SIZE                                                                 ║
║  ├─ Primary Implants (Hip + Knee):      484,918 procedures             ║
║  ├─ Revision Procedures:                 34,909 procedures             ║
║  └─ Total Relevant Procedures:          519,827 procedures             ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  EXPECTED INFECTION INDEX (EII) - Primary Implants Only                      ║
║  ├─ Conservative (1.0%):                  4,849 infections             ║
║  ├─ Mid-range (1.5%):                     7,273 infections             ║
║  └─ Upper bound (2.0%):                   9,698 infections             ║
╠══════════════════════════════════════════════════════════════

---
# Data Export

Export analysis results for further use.

In [66]:
# Export Deliverables
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d')

# 1. Full Hospital Ranking (CSV)
export_cols = [
    'IK', 'Name', 'Ort', 'Postleitzahl', 'Bundesland',
    'hip_primary', 'hip_revision', 'knee_primary', 'knee_revision',
    'total_procedures', 'total_primary', 'total_revision', 'revision_rate',
    'EII_low', 'EII_mid', 'EII_high',
    'antibiotic_mention_count', 'has_relevant_dept', 'is_vollversorger',
    'opportunity_score'
]
hospital_ranked[export_cols].to_csv(f'hospital_ranking_{timestamp}.csv', index=False)
print(f"✅ Exported: hospital_ranking_{timestamp}.csv")

# 2. Tiered Target Lists (Excel)
with pd.ExcelWriter(f'target_hospitals_{timestamp}.xlsx') as writer:
    tier1[export_cols].to_excel(writer, sheet_name='Tier1_Priority', index=False)
    tier2[export_cols].to_excel(writer, sheet_name='Tier2_Secondary', index=False)
    tier3[export_cols].to_excel(writer, sheet_name='Tier3_Reference', index=False)
print(f"✅ Exported: target_hospitals_{timestamp}.xlsx")

# 3. Regional Summary (Excel)
regional_df.to_excel(f'regional_summary_{timestamp}.xlsx', index=False)
print(f"✅ Exported: regional_summary_{timestamp}.xlsx")

# 4. Multi-year Trends (Excel)
trend_df.to_excel(f'multiyear_trends_{timestamp}.xlsx', index=False)
print(f"✅ Exported: multiyear_trends_{timestamp}.xlsx")

print("\n📁 All exports completed successfully.")

✅ Exported: hospital_ranking_20260203.csv
✅ Exported: target_hospitals_20260203.xlsx
✅ Exported: regional_summary_20260203.xlsx
✅ Exported: multiyear_trends_20260203.xlsx

📁 All exports completed successfully.


---
# Verification Checklist

| # | Requirement | Status |
|---|-------------|--------|
| 1 | All cells run without errors | ☐ |
| 2 | Section 4 establishes 2023 OPS baseline FIRST | ✅ |
| 3 | Section 5 validates with multi-year trend (2018-2023) | ✅ |
| 4 | Multi-year analysis answers: stable or growing market? | ✅ |
| 5 | ICD analysis includes exclusions (T84, Z96.6, M00), clearly SECONDARY | ✅ |
| 6 | TAM/Market sizing appears BEFORE hospital breakdown | ✅ |
| 7 | Hospital characteristics folded into Section 9 | ✅ |
| 8 | OAU proxy has prominent caveats | ✅ |
| 9 | EII uses 1-2% infection rate | ✅ |
| 10 | Section 12 lists all unanswerable briefing requirements | ✅ |
| 11 | Destatis age data noted as backlog option | ✅ |
| 12 | Flow: 2023 baseline → Trend validation → TAM → Hospital → Regional | ✅ |